In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
import pandas as pd
import numpy as np

In [3]:
!unzip /kaggle/input/quora-question-pairs/train.csv.zip 

Archive:  /kaggle/input/quora-question-pairs/train.csv.zip
  inflating: train.csv               


In [ ]:
!ls -l

In [4]:
data = pd.read_csv('train.csv')

In [ ]:
data

In [5]:
# Ref: https://hub.packtpub.com/use-tensorflow-and-nlp-to-detect-duplicate-quora-questions-tutorial/
# adding new features

data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2

In [6]:
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split(' ')))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split(' ')))

# common words
data['common_words'] = data.apply(
    lambda x: 
    len(set(str(x['question1']).lower().split())
        .intersection(
            set(str(x['question2']).lower().split()))), 
    axis=1)

In [7]:
fs_1 = ['len_q1', 'len_q2', 'diff_len', 'len_char_q1', 
        'len_char_q2', 'len_word_q1', 'len_word_q2',     
        'common_words']

In [ ]:
!pip install fuzzywuzzy python-Levenshtein

In [8]:
# using fuzzywuzzy package to find levenshtein distance between question pairs
from fuzzywuzzy import fuzz

In [ ]:
# example of calculating fuzzy number
fuzz.QRatio("How can I start an online shopping (e-commerce) website?", "Which web technology is best suitable for building a big E-Commerce website?")


In [ ]:
fuzz.partial_ratio("Why did Trump win the Presidency?", 
                   "How did Donald Trump win the 2016 Presidential Election")


In [ ]:
fuzz.partial_ratio("How can I start an online shopping (e-commerce) website?", 
                   "Which web technology is best suitable for building a big E-Commerce website?")


In [9]:
def apply_fuzz(data, column_name, function):
    data[column_name] = data.apply(
        lambda x: function(str(x['question1']), str(x['question2'])), axis=1
    )

In [10]:
apply_fuzz(data, 'fuzz_ratio', fuzz.QRatio)
apply_fuzz(data, 'fuzz_WRatio', fuzz.WRatio)
apply_fuzz(data, 'fuzz_partial_ratio', fuzz.partial_ratio)
apply_fuzz(data, 'fuzz_partial_token_set_ratio', fuzz.partial_token_set_ratio)
apply_fuzz(data, 'fuzz_partial_token_sort_ratio', fuzz.partial_token_sort_ratio)
apply_fuzz(data, 'fuzz_token_set_ratio', fuzz.token_set_ratio)
apply_fuzz(data, 'fuzz_token_sort_ratio', fuzz.token_sort_ratio)

In [19]:
fs_2 = ['fuzz_ratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio']

In [ ]:
data

In [ ]:
# tf-idf implementation
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import deepcopy

tfv_q1 = TfidfVectorizer(
    min_df=3, 
    max_features=None, 
    strip_accents='unicode', 
    analyzer='word', 
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2), 
    use_idf=1, 
    smooth_idf=1, 
    sublinear_tf=1,
    stop_words='english')

tfv_q2 = TfidfVectorizer(
    min_df=3, 
    max_features=None, 
    strip_accents='unicode', 
    analyzer='word', 
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2), 
    use_idf=1, 
    smooth_idf=1, 
    sublinear_tf=1,
    stop_words='english')


In [ ]:
q1_tfidf = tfv_q1.fit_transform(data.question1.fillna(""))
q2_tfidf = tfv_q2.fit_transform(data.question2.fillna(""))

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_q1 = TruncatedSVD(n_components=180)
svd_q2 = TruncatedSVD(n_components=180)

In [ ]:
question1_vectors = svd_q1.fit_transform(q1_tfidf)
question2_vectors = svd_q2.fit_transform(q2_tfidf)

In [ ]:
from scipy import sparse
# obtain features by stacking the sparse matrices together
fs3_1 = sparse.hstack((q1_tfidf, q2_tfidf))

In [ ]:
tfv = TfidfVectorizer(min_df=3, 
                      max_features=None, 
                      strip_accents='unicode', 
                      analyzer='word', 
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 2), 
                      use_idf=1, 
                      smooth_idf=1, 
                      sublinear_tf=1,
                      stop_words='english')

In [ ]:
# combine questions and calculate tf-idf
q1q2 = data.question1.fillna("") 
q1q2 += " " + data.question2.fillna("")
fs3_2 = tfv.fit_transform(q1q2)

In [ ]:
fs3_3 = np.hstack((question1_vectors, question2_vectors))

In [ ]:
import gensim.downloader as api

model = api.load('word2vec-google-news-300')


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = set(stopwords.words('english'))

def sent2vec(s, model): 
    M = []
    words = word_tokenize(str(s).lower())
    for word in words:
    #It shouldn't be a stopword
        if word not in stop_words:
            #nor contain numbers
            if word.isalpha():
                #and be part of word2vec
                if word in model:
                    M.append(model[word])
    M = np.array(M)
    if len(M) > 0:
        v = M.sum(axis=0)
        return v / np.sqrt((v ** 2).sum())
    else:
        return np.zeros(300)

In [ ]:
w2v_q1 = np.array([sent2vec(q, model) 
                   for q in data.question1])
w2v_q2 = np.array([sent2vec(q, model) 
                   for q in data.question2])

In [ ]:
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

In [ ]:
data['cosine_distance'] = [cosine(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['cityblock_distance'] = [cityblock(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['jaccard_distance'] = [jaccard(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['canberra_distance'] = [canberra(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['euclidean_distance'] = [euclidean(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['minkowski_distance'] = [minkowski(x,y,3) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['braycurtis_distance'] = [braycurtis(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]

In [ ]:
fs4_1 = ['cosine_distance', 'cityblock_distance', 
         'jaccard_distance', 'canberra_distance', 
         'euclidean_distance', 'minkowski_distance',
         'braycurtis_distance']


In [ ]:
w2v = np.hstack((w2v_q1, w2v_q2))

In [ ]:
def wmd(s1, s2, model):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

In [ ]:
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2'], model), axis=1)
model.init_sims(replace=True) 
data['norm_wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2'], model), axis=1)
fs4_2 = ['wmd', 'norm_wmd']

In [13]:
import gc
import psutil
# del([tfv_q1, tfv_q2, tfv, q1q2, 
#      question1_vectors, question2_vectors, svd_q1, 
#      svd_q2, q1_tfidf, q2_tfidf])
# del([w2v_q1, w2v_q2])
# del([model])
gc.collect()
psutil.virtual_memory()

svmem(total=16787144704, available=15253102592, percent=9.1, used=1255366656, free=9576894464, active=1332031488, inactive=5276549120, buffers=1048141824, cached=4906741760, shared=999424, slab=492085248)

In [14]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [17]:
fs_1+fs_2

['len_q1',
 'len_q2',
 'diff_len',
 'len_char_q1',
 'len_char_q2',
 'len_word_q1',
 'len_word_q2',
 'common_words',
 'fuzz_qratio',
 'fuzz_WRatio',
 'fuzz_partial_ratio',
 'fuzz_partial_token_set_ratio',
 'fuzz_partial_token_sort_ratio',
 'fuzz_token_set_ratio',
 'fuzz_token_sort_ratio']

In [21]:
scaler = StandardScaler()
y = data.is_duplicate.values
y = y.astype('float32').reshape(-1, 1)
X = data[fs_1+fs_2]
X = X.replace([np.inf, -np.inf], np.nan).fillna(0).values
X = scaler.fit_transform(X)
# X = np.hstack((X, fs3_3))

In [22]:
np.random.seed(42)
n_all, _ = y.shape
idx = np.arange(n_all)
np.random.shuffle(idx)
n_split = n_all // 10
idx_val = idx[:n_split]
idx_train = idx[n_split:]
x_train = X[idx_train]
y_train = np.ravel(y[idx_train])
x_val = X[idx_val]
y_val = np.ravel(y[idx_val])

In [23]:
# logres = linear_model.LogisticRegression(C=0.1, 
#                                  solver='sag', max_iter=1000)
# logres.fit(x_train, y_train)
# lr_preds = logres.predict(x_val)
# log_res_accuracy = np.sum(lr_preds == y_val) / len(y_val)
# print("Logistic regr accuracy: %0.3f" % log_res_accuracy)

Logistic regr accuracy: 0.663


In [24]:
params = dict()
params['objective'] = 'binary:logistic'
params['eval_metric'] = ['logloss', 'error']
params['eta'] = 0.02
params['max_depth'] = 4
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_val, label=y_val)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 5000, watchlist, 
                early_stopping_rounds=50, verbose_eval=100)
xgb_preds = (bst.predict(d_valid) >= 0.5).astype(int)
xgb_accuracy = np.sum(xgb_preds == y_val) / len(y_val)
# print("Xgb accuracy: %0.3f" % xgb_accuracy)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:528: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning


[0]	train-logloss:0.68786	train-error:0.30518	valid-logloss:0.68786	valid-error:0.30510
[100]	train-logloss:0.52635	train-error:0.29646	valid-logloss:0.52787	valid-error:0.29560
[200]	train-logloss:0.50768	train-error:0.29134	valid-logloss:0.50968	valid-error:0.29095
[300]	train-logloss:0.50083	train-error:0.28722	valid-logloss:0.50299	valid-error:0.28668
[400]	train-logloss:0.49617	train-error:0.28455	valid-logloss:0.49867	valid-error:0.28413
[500]	train-logloss:0.49214	train-error:0.28102	valid-logloss:0.49506	valid-error:0.28126
[600]	train-logloss:0.48939	train-error:0.27917	valid-logloss:0.49264	valid-error:0.28010
[700]	train-logloss:0.48668	train-error:0.27705	valid-logloss:0.49026	valid-error:0.27856
[800]	train-logloss:0.48457	train-error:0.27540	valid-logloss:0.48855	valid-error:0.27710
[900]	train-logloss:0.48291	train-error:0.27374	valid-logloss:0.48722	valid-error:0.27592
[1000]	train-logloss:0.48130	train-error:0.27221	valid-logloss:0.48593	valid-error:0.27522
[1100]	trai

In [34]:
test_df = pd.read_csv("/kaggle/input/quora-question-pairs/test.csv.zip")
submit_df = pd.read_csv("/kaggle/input/quora-question-pairs/sample_submission.csv.zip")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
test_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
3563470,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
3563471,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
3563472,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
3563473,2345794,What were the best and worst things about publ...,What are the best and worst things examination...


In [36]:
submit_df

,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
2345791,2345791,1
2345792,2345792,1
2345793,2345793,1
2345794,2345794,1
